In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pdfplumber

In [28]:
with pdfplumber.open("Invoice.pdf") as pdf_file:
    data = []    
    for i in range(0, len(pdf_file.pages)):
        page_data = pdf_file.pages[i]
        data.append(page_data.extract_text())

In [31]:
Incidentals = []
for i in range(0,len(data)):
    text_list = data[i].split('\n') # split lines using \n escape char
    Ientries = [e for e in text_list if "Incid" in e]
    Incidentals = Incidentals + Ientries
    '''
    for j in range(0,len(text_list)):
        text = text_list[j]
        incid_fee_entries = [e for e in text_list if ]
        '''

In [25]:
data[0].split('\n')[0]

'8/2/2020 Invoice'

In [32]:
Incidentals

['Incid. Stud.Soc.:ES Concrete Canoe Team-FT           $      1.37                ',
 'Incid. Stud.Soc.:ES Concrete Toboggan Tm-FT          $      1.75                ',
 'Incid. Stud.Soc.:ES Engineers in Action-FT           $      0.75                ',
 'Incid. Stud.Soc.:ES Eng. w/o Borders -FT             $      1.50                ',
 'Incid. Stud.Soc.:ES Formula SAE Racing Tm-FT         $      2.75                ',
 'Incid. Stud.Soc.:ES Hum Powr Veh Desgn-FT            $      2.48                ',
 'Incid. Stud.Soc.:ES Robotics fr Space Expl-FT        $      2.00                ',
 'Incid. Stud.Soc.:ES Skule Endowmt Fund-FT            $     50.00                ',
 'Incid. Stud.Soc.:ES Skule Night-FT                   $      1.95                ',
 'Incid. Stud.Soc.:ES UofT Aerospace Team-FT           $      3.35                ',
 'Incid. Stud.Soc.:ES Robotics Assoc-FT                $      2.00                ',
 'Incid. Stud.Soc.:UTSU Bike Chain                     $      0.5